In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

from pyomo.opt import TerminationCondition

# Workshop on Optimization Techniques for Data Science in Python and Julia
 
 Fields Institute, 2019

Videos available at:
- http://www.fields.utoronto.ca/activities/18-19/optimization

NOtes available at: 
- https://github.com/bkamins/Workshop-on-Optimization-Techniques

# Lesson 1
### Model construction (specification)

In [40]:
model = ConcreteModel(name="Chess set 1")
model.xs = Var(within=NonNegativeIntegers) #NonNegativeReals
model.xl = Var(within=NonNegativeIntegers) #NonNegativeReals
model.obj = Objective(expr=5*model.xs + 20*model.xl, sense=maximize)
model.latehours = Constraint(expr= 3*model.xs + 2*model.xl <= 52*160) #52 weeks
model.boxwood = Constraint(expr= 1*model.xs + 3*model.xl <= 52*200)

### Model solving 
There are many solvers... You should select it according to the type of problem you have. 
There are open source solvers. 

`glpk` is able to solve integer problems. It just selects a different algorithm. 

In [37]:
solver = SolverFactory("glpk")  #
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: 69330.0
  Upper bound: 69330.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.08741021156311035
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [38]:
print(value(model.xs), value(model.xl))

2.0 3466.0


In [9]:
#To check that the model is as expected
model.obj.pprint()
model.latehours.pprint()
model.boxwood.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 5*xs + 20*xl


Other way of writing the model in python - the advantage is that you have more flexibility and power to do it. 

In [43]:
def objective(n):
    return 5*n.xs + 20*n.xl

def latehours(n):
    return 3*n.xs + 2*n.xl <= 160

def boxwood(n):
    return 1*n.xs + 3*n.xl <= 200

constraints = [latehours, boxwood]

def constraint(n, idx):
    return constraints[idx](n)


def chess_set(onlyint):
    typ = NonNegativeIntegers if onlyint else NonNegativeReals
    model = ConcreteModel(name="Chess set")
    model.xs=Var(within=typ)
    model.xl=Var(within=typ)
    model.obj = Objective(rule=objective, sense=maximize) #rule is different to expression. Rule is a function. 
    model.constr = Constraint([0,1], rule=constraint)
    solver = SolverFactory("glpk")
    solver.solve(model)
    return(model)
    

In [44]:
res= chess_set(False)
res.pprint()

1 Set Declarations
    constr_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}

2 Var Declarations
    xl : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 66.6666666666667 :  None : False : False : NonNegativeReals
    xs : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5*xs + 20*xl

1 Constraint Declarations
    constr : Size=2, Index=constr_index, Active=True
        Key : Lower : Body        : Upper : Active
          0 :  -Inf : 3*xs + 2*xl : 160.0 :   True
          1 :  -Inf :   xs + 3*xl : 200.0 :   True

5 Declarations: xs xl obj constr_index constr


In [45]:
res= chess_set(True)
res.pprint()

1 Set Declarations
    constr_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}

2 Var Declarations
    xl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  66.0 :  None : False : False : NonNegativeIntegers
    xs : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   2.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5*xs + 20*xl

1 Constraint Declarations
    constr : Size=2, Index=constr_index, Active=True
        Key : Lower : Body        : Upper : Active
          0 :  -Inf : 3*xs + 2*xl : 160.0 :   True
          1 :  -Inf :   xs + 3*xl : 200.0 :   True

5 Declarations: xs xl obj constr_index constr


- https://neos-guide.org/optimization-tree
- https://documentation.aimms.com/platform/solvers/solvers.html#available-solvers

# Lesson 2
## Problem 1

In [100]:
# Constants

demand = [30.0, 15.0, 15.0, 25.0, 33.0, 40.0,
          45.0, 45.0, 26.0, 14.0, 25.0, 30.0]

cost_normal = 32*1000
cost_overtime = 40*1000
cost_store = 5*1000 # *0 #if storing is free, we have more than one optimal solution
capacity = 30
n=len(demand)

In [101]:
model = ConcreteModel(name="Bicycle")
model.time = RangeSet(1,n) #define set
model.store_time = RangeSet(0,n)
model.prod_normal = Var(model.time, bounds=(0,capacity)) #variable in terms of model.time or indexed in model.time
model.prod_overtime = Var(model.time, bounds=(0,15)) #define lower and upper bound
model.store = Var(model.store_time, within=NonNegativeReals)

In [102]:
# defined dynamically. 
model.obj = Objective(expr=sum(cost_normal*model.prod_normal[i]+
                              cost_overtime*model.prod_overtime[i]+
                              cost_store*model.store[i] for i in model.time),
                     sense=minimize)

def time_constraint(model, t):
    inflow = model.prod_normal[t] + model.prod_overtime[t] + model.store[t-1]
    outflow = demand[t-1] + model.store[t]
    return inflow == outflow

model.constr = Constraint(model.time, rule=time_constraint)

model.store[0].fix(2.0)

In [103]:
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)
model.pprint()


Problem: 
- Name: unknown
  Lower bound: 11247000.0
  Upper bound: 11247000.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 37
  Number of nonzeros: 48
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.09215831756591797
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

2 RangeSet Declarations
    store_time : Dimen=1, Size=13, Bounds=(0, 12)
        Key  : Finite : Members
        None :   True :  [0:12]
    time : Dimen=1, Size=12, Bounds=(1, 12)
        Key  : Finite : Members
        None :   True :  [1:12]

3 Var Declarations
    prod_normal : Size=12, Index=time
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  28.0 :    30 : False : False :  Reals
          2 :     0 :  15.0 :    30 : False : False :  Reals
          3 :     0 :  15.

In [104]:
print("demand\tnormal\tover\tstore")
for t in range(1,13):
    print("{:4}\t{:4}\t{:4}\t{:4}".format(demand[t-1],
          value(model.prod_normal[t]),
          value(model.prod_overtime[t]), 
          value(model.store[t])))

demand	normal	over	store
30.0	28.0	 0.0	 0.0
15.0	15.0	 0.0	 0.0
15.0	15.0	 0.0	 0.0
25.0	28.0	 0.0	 3.0
33.0	30.0	 0.0	 0.0
40.0	30.0	10.0	 0.0
45.0	30.0	15.0	 0.0
45.0	30.0	15.0	 0.0
26.0	26.0	 0.0	 0.0
14.0	14.0	 0.0	 0.0
25.0	25.0	 0.0	 0.0
30.0	30.0	 0.0	 0.0


In [105]:
results.solver.termination_condition

<TerminationCondition.optimal: 'optimal'>

## Problem 2

In [87]:
model = ConcreteModel(name="Infeasible")
model.x = Var(within=NonNegativeReals)
model.obj = Objective(expr= model.x, sense=maximize)
model.constr = Constraint(expr= model.x <= -1)
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 2
  Number of nonzeros: 2
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.2057340145111084



In [88]:
model.pprint()

1 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize :          x

1 Constraint Declarations
    constr : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  -1.0 :   True

3 Declarations: x obj constr


In [93]:
from pyomo.opt import TerminationCondition

In [96]:
results.solver.termination_condition

<TerminationCondition.infeasible: 'infeasible'>

In [97]:
model = ConcreteModel(name="Unbounded")
model.x = Var(within=NonNegativeReals)
model.obj = Objective(expr= model.x, sense=maximize)
model.constr = Constraint(expr= model.x >= -1)
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 2
  Number of nonzeros: 2
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: unbounded
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0866537094116211



In [98]:
model.pprint()

1 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize :          x

1 Constraint Declarations
    constr : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -1.0 :    x :  +Inf :   True

3 Declarations: x obj constr


# Lesson 3